In [ ]:
import re
from textblob import TextBlob
from emoji import demojize
def clean_text(text):
    if isinstance(text, str):
        
        text = demojize(text)
        
        text = re.sub(r'http\S+', '', text) # Remove URLs
        text = re.sub(r'\s+', ' ', text, flags=re.I) # substitute multiple space to single space
        
        # text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
        # text = re.sub(r'#\w+', '', text) #Remove hashtags
        text = text.lower()
        # Handle self-censored words and emojis as needed
        
        # misspellings?
        text_blob = TextBlob(text)
        text = str(text_blob.correct())
        
        censored_words = {
            'sh*t': 'shit',
            'f*ck': 'fuck',
            'pu$$y': 'pussy',
        }
        for censored, uncensored in censored_words.items():
            text = text.replace(censored, uncensored)
        text = re.sub(r'[^a-zA-Z0-9\s?!.]', '', text)
        
        pass
    
    else:
        return ""
    return text

cleaned = df['comment_text'].apply(clean_text)
cleaned